In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer

from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   

def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [3]:
df = pd.read_json('ai_papers.json')

In [4]:
# Preprocessing the dataset
df['abstract'] = df['abstract'].apply(clean_txt)

In [5]:
# Looking at the cleaned dataset
df.head()

,id,title,category,abstract,version
0,0704.0047,Intelligent location of simultaneously active ...,cs.NE cs.AI,intelligent acoustic emission locator describe...,"Sun, 1 Apr 2007 13:06:50 GMT"
1,0704.0050,Intelligent location of simultaneously active ...,cs.NE cs.AI,part describe intelligent acoustic emission lo...,"Sun, 1 Apr 2007 18:53:13 GMT"
2,0704.0304,The World as Evolving Information,cs.IT cs.AI math.IT q-bio.PE,paper discuss benefit describe world informati...,"Wed, 13 Oct 2010 19:49:16 GMT"
3,0704.0671,Learning from compressed observations,cs.IT cs.LG math.IT,problem statistical learn construct predictor ...,"Thu, 5 Apr 2007 02:57:15 GMT"
4,0704.0954,Sensor Networks with Random Links: Topology De...,cs.IT cs.LG math.IT,sensor network practice communication among se...,"Fri, 6 Apr 2007 21:58:52 GMT"


In [20]:
# Count vectorizer method
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

count = count_vectorizer.fit_transform((df['abstract'])) #fitting and transforming the vector
count

<91496x78832 sparse matrix of type '<class 'numpy.int64'>'
	with 6652009 stored elements in Compressed Sparse Row format>

In [21]:
type(count)

scipy.sparse.csr.csr_matrix

In [7]:
user_query = "face recognition deep learning convolutional neural networks"
user_query = clean_txt(user_query)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
user_count = count_vectorizer.transform([user_query])
cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count)

In [22]:
count = np.load('count_matrix.npy', allow_pickle=True)

In [24]:
from scipy.sparse import csr_matrix

D = csr_matrix(count.all())

In [25]:
D

<91496x78832 sparse matrix of type '<class 'numpy.int64'>'
	with 6652009 stored elements in Compressed Sparse Row format>

In [20]:
output2 = list(cos_similarity_countv)


In [21]:
def get_recommendation(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['Title', 'score'])
    count = 0
    for i in top:
        recommendation.at[count, 'Title'] = df['title'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation

In [22]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]
list_scores = [output2[i][0][0] for i in top]
get_recommendation(top, df, list_scores)

,Title,score
0,Universality of Deep Convolutional Neural Netw...,0.65809
1,Deep Recurrent Convolutional Neural Network: I...,0.627785
2,Deep Learning for Computational Chemistry,0.601493
3,Implementation of Training Convolutional Neura...,0.594322
4,A Unified Framework of Deep Neural Networks by...,0.58125
5,Applications of Deep Neural Networks,0.580544
6,DLL: A Blazing Fast Deep Neural Network Library,0.574722
7,Deep Reinforcement Learning: An Overview,0.570925
8,Recent Advances in Convolutional Neural Networks,0.564218
9,Convex Formulation of Overparameterized Deep N...,0.562286
